In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# https://www.kaggle.com/datasets/abdelrahmankhalil/100000-uk-used-car-data-set

In [20]:
df = pd.read_csv('unzipped_archive/bmw.csv')

# Валидационная выборка

In [28]:
# train = 60%, validate = 20%, test = 20%

# сначала определяем train = 60%:

train, test = train_test_split(df, train_size = 0.6, random_state = 42)

In [30]:
len(train) / len(df)

0.5999443465355718

In [31]:
len(test) / len(df)

0.4000556534644282

In [32]:
# теперь определяем валидационную выборку и test - как 50% на 50% (от теста)
 
val, test = train_test_split(test, train_size = 0.5, random_state = 42)

In [33]:
len(val) / len(df)

0.19998144884519062

In [34]:
len(test) / len(df)

0.20007420461923756

# Список фичей

In [ ]:
# Разновидности фичей:
    
# непрерывные (числовые) фичи = ['year', 'mileage', 'tax', 'mpg', 'engineSize']

# и 

# категориальные фичи = ['model', 'transmission', 'fuelType']

In [35]:
# все наши признаки: (включая целевой признак - его надо убрать из фичей)

train.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [36]:
# список фичей сохраняем в переменную Х (x-большое):

X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']

In [37]:
# целевой признак сохраняем в переменную y (y-маленькое):

y = ['price']

In [38]:
# определяем категориальные фичи:

cat_features = ['model', 'transmission', 'fuelType']